In [47]:
import pandas as pd
import utils

# figures library

import matplotlib.pyplot as plt
plt.rcParams.update({'pdf.fonttype': 42})

import seaborn as sns
sns.set(style='whitegrid',context="poster")

In [48]:
def label_gap(gap):
    if gap > 0:
        return "state-of-art"
    if gap < 0:
        return "sa"
    return "equal"

def label_nuances(gap): 
    if gap > 1:
        return ">1"
    if gap < -1:
        return "< -1"
    if gap > 0:
        return " 0 to 1"
    if gap < 0:
        return "-1 to 0"
    return "equal"

In [49]:
def create_df_for_gap(uc,selector):
    sa = pd.read_csv(f"../processed-datasets/sa-1-milion/sa-uc{uc}-1-milion.csv", index_col=0)
    exact = pd.read_csv(f"../processed-datasets/exact-methods/exact-methods-uc-{uc}.csv", index_col=0)
    lns = pd.read_csv(f"../processed-datasets/lns-results/lns-uc-{uc}.csv", index_col=0)

    df = pd.merge(sa,exact,on="instance")
    df = pd.merge(df,lns,on="instance")
    df["state_of_art"] = df.apply(lambda row: utils.state_of_the_art([row["ilp"],row["cp"],row["LNS"]]),axis=1)
    df["gap"] = df.apply(lambda row: utils.calculate_gap(row["total_cost_min"],row["state_of_art"]),axis=1)
    df["gap_ilp"] = df.apply(lambda row: utils.calculate_gap(row["total_cost_min"],row["ilp"]),axis=1)
    df["gap_cp"] = df.apply(lambda row: utils.calculate_gap(row["total_cost_min"],row["cp"]),axis=1)
    df["gap_lns"] = df.apply(lambda row: utils.calculate_gap(row["total_cost_min"],row["LNS"]),axis=1)
    df["label_gap"] = df.apply(lambda row: label_gap(row["gap"]),axis=1)
    df["label_nuances"] = df.apply(lambda row: label_nuances(row["gap"]),axis=1)
    df["n_jobs"] = df.apply(lambda row: utils.number_of_jobs(row["instance"]), axis=1)
    df["size"] = df.apply(lambda row: utils.instance_size(row["instance"]), axis=1)
    #if uc == 2:
    #    df = df.query("instance != [102, 103, 107, 110, 111, 114, 84, 88, 92, 94, 95, 98]")
    df = df.query(f"instance > {selector}")
    print(f"==== UC {uc} === GAP GENERAL ====== ")
    display(df["gap"].mean())
    display(df["gap"].std())
    display(df[["size","gap"]].groupby("size").mean())
    display(df[["size","gap"]].groupby("size").std())
    return df

In [50]:
selector = 40
df_1 = create_df_for_gap(1,selector)[["instance", "total_cost_min", "ilp", "cp", "LNS", "gap", "n_jobs","size","label_gap","label_nuances"]].query(f"instance > {selector}").copy().rename(columns={"total_cost_min":"SA", "ilp":"ILP", "cp":"CP"})
df_2 = create_df_for_gap(2,selector)[["instance", "total_cost_min", "ilp", "cp", "LNS", "gap", "n_jobs","size","label_gap","label_nuances"]].query(f"instance > {selector}").copy().rename(columns={"total_cost_min":"SA", "ilp":"ILP", "cp":"CP"})
df_3 = create_df_for_gap(3,selector)[["instance", "total_cost_min", "ilp", "cp", "LNS", "gap", "n_jobs","size","label_gap","label_nuances"]].query(f"instance > {selector}").copy().rename(columns={"total_cost_min":"SA", "ilp":"ILP", "cp":"CP"})

==== UC 1 === GAP GENERAL ====== 


-3.5315962499999998

8.074514493794167

,gap
size,
Large,-6.846648
Medium,-0.216545


,gap
size,
Large,10.384725
Medium,1.296958


...

==== UC 3 === GAP GENERAL ====== 


-4.8901562499999995

5.762336603484664

,gap
size,
Large,-8.722897
Medium,-1.057415


,gap
size,
Large,5.852258
Medium,1.696581


In [51]:
# selector = 40
# cols = ["instance", "total_cost_min", "ilp", "cp", "LNS", "gap_cp", "gap_ilp", "gap_lns","n_jobs","size","label_gap","label_nuances"]
# df_1_gappy = create_df_for_gap(1,selector)[cols].query(f"instance > {selector}").copy().rename(columns={"total_cost_min":"SA", "ilp":"ILP", "cp":"CP"})
# df_2_gappy = create_df_for_gap(2,selector)[cols].query(f"instance > {selector}").copy().rename(columns={"total_cost_min":"SA", "ilp":"ILP", "cp":"CP"})
# df_3_gappy = create_df_for_gap(3,selector)[cols].query(f"instance > {selector}").copy().rename(columns={"total_cost_min":"SA", "ilp":"ILP", "cp":"CP"})

In [52]:
fig,(ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(10,4),sharey=True)
order = ["Medium","Large"]
palette = [ "white", "white"] #[sns.color_palette()[0],sns.color_palette()[1]]

sns.boxplot(ax=ax1, x='size', y='gap', data=df_1, order=order,palette=palette, hue="size", linecolor="black")#  boxprops=dict(facecolor='#FFFFFF', alpha=1, zorder = 1)
ax1.set_ylabel('Gap [%]', )
ax1.set_xlabel(None, )
#ax1.set_ylim(-60,20)
ax1.set_title("UC-1",)
ax1.tick_params(axis='both', which='major', )

#df_2 = df_2.query("instance != [110, 114, 111, 107, 103, 102]")
sns.boxplot(ax=ax2, x='size', y='gap', data=df_2,  order=order,palette=palette, hue="size",linecolor="black")#  boxprops=dict(facecolor='#FFFFFF', alpha=1, zorder = 1)
#sns.swarmplot(ax=ax, x='size', y='gap', data=df, order=order, s=7, palette=palette,alpha = 0.75)
ax2.set_ylabel(None, )
ax2.set_xlabel(None, )
#ax2.set_ylim(-60,15)
ax2.set_title("UC-2",)
ax2.tick_params(axis='both', which='major', )

sns.boxplot(ax=ax3, x='size', y='gap', data=df_3,  order=order,palette=palette, hue="size",linecolor="black")#  boxprops=dict(facecolor='#FFFFFF', alpha=1, zorder = 1)
#sns.swarmplot(ax=ax, x='size', y='gap', data=df, order=order, s=7, palette=palette,alpha = 0.75)
ax3.set_ylabel(None, )
ax3.set_xlabel(None, )
# ax3.set_ylim(-60,15)
ax3.set_title("UC-3",)
ax3.tick_params(axis='both', which='major', )

plt.tight_layout()
fig.figure.savefig(f"sa-gap-1-milion.pdf",)
plt.show()

/var/folders/tm/vk6plqp1543cptt9z15vkbk40000gn/T/ipykernel_3415/2386152893.py:31: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [53]:
df_2[["size","label_nuances","instance"]].groupby(["size","label_nuances"]).count()

instance
size   label_nuances          
Large  < -1                 37
       >1                    2
       equal                 1
Medium  0 to 1              10
       -1 to 0               5
       < -1                  2
       >1                   14
       equal                 9

In [54]:
df_3[["size","label_nuances","instance"]].groupby(["size","label_nuances"]).count()

instance
size   label_nuances          
Large  -1 to 0               1
       < -1                 39
Medium  0 to 1               2
       -1 to 0               9
       < -1                 12
       >1                    1
       equal                16